In [2]:
import pandas as pd
import numpy as np

In [4]:
projects = pd.read_csv('nrao_projects_use.csv')

In [5]:
projects.head(10)

,project_code,project_title,project_abstract,fs_type,science_category,science_keyword,band,target,raw_text,standardized_text,no_sw_text,lemmatized_sw_text,lemmatized_no_sw_text
0,2021.1.01616.L,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,We propose the first ever statistical survey o...,line,Galaxy evolution,"Surveys of galaxies, Galaxy groups and clusters",6,1,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,jelly survey of nearby jellyfish and ram press...,jelly survey nearby jellyfish ram pressure str...,jelly survey of nearby jellyfish and ram press...,jelly survey nearby jellyfish ram pressure str...
1,2022.1.01077.L,A SPectroscopic survey of biased halos In the ...,We propose to obtain deep ALMA 1.2mm mosaic ob...,line,Galaxy evolution,"Sub-mm Galaxies (SMG), High-z Active Galactic ...",6,1,A SPectroscopic survey of biased halos In the ...,a spectroscopic survey of biased halos in the ...,spectroscopic survey biased halos reionization...,a spectroscopic survey of biased halo in the r...,spectroscopic survey bias halos reionization e...
2,2016.1.00324.L,ASPECS: The ALMA SPECtral line Survey in the U...,ASPECS represents an unparalleled three-dimens...,line,Galaxy evolution,Lyman Break Galaxies (LBG),3,1,ASPECS: The ALMA SPECtral line Survey in the U...,aspecs the spectral line survey in the udf an ...,aspecs spectral line survey udf large program ...,aspecs the spectral line survey in the udf an ...,aspecs spectral line survey udf large program ...
3,2022.1.00875.L,The ALMA Disk-Exoplanet C/Onnection,Protoplanetary disks set the initial compositi...,line,Disks and planet formation,"Disks around low-mass stars, Exo-planets",7,1,The ALMA Disk-Exoplanet C/Onnection. Protoplan...,the disk exoplanet c onnection protoplanetary ...,disk exoplanet c onnection protoplanetary disk...,the disk exoplanet c onnection protoplanetary ...,disk exoplanet c onnection protoplanetary disk...
4,2017.1.01355.L,ALMA-IMF: ALMA transforms our view of the orig...,Studying massive protoclusters is an absolute ...,line,ISM and star formation,"High-mass star formation, Low-mass star formation",6,1,ALMA-IMF: ALMA transforms our view of the orig...,imf transforms our view of the origin of stell...,imf transforms view origin stellar masses stud...,imf transform our view of the origin of stella...,imf transforms view origin stellar mass study ...
5,2016.1.00484.L,Small-Scale Substructures in Protoplanetary Disks,We propose a Large Program designed to charact...,line,Disks and planet formation,Disks around low-mass stars,6,1,Small-Scale Substructures in Protoplanetary Di...,small scale substructures in protoplanetary di...,small scale substructures protoplanetary disks...,small scale substructure in protoplanetary dis...,small scale substructure protoplanetary disk p...
6,2021.1.01123.L,exoALMA,Detecting planets embedded in their natal prot...,line,Disks and planet formation,"Disks around low-mass stars, Exo-planets",7,1,exoALMA. Detecting planets embedded in their n...,exo detecting planets embedded in their natal ...,exo detecting planets embedded natal protoplan...,exo detect planet embed in their natal protopl...,exo detect planet embed natal protoplanetary d...
7,2019.1.01634.L,REBELS: An ALMA Large Program to Discover the ...,REBELS will construct the first large statisti...,line,Galaxy evolution,"Lyman Break Galaxies (LBG), Galaxy structure &...",6,1,REBELS: An ALMA Large Program to Discover the ...,rebels an large program to discover the most l...,rebels large program discover luminous cii oii...,rebel an large program to discover the most lu...,rebel large program discover luminous cii oiii...
8,2019.1.00261.L,Early Planet Formation in Embedded Disks,We propose an ALMA Large Program to investigat...,line,Disks and planet formation,Disks around low-mass stars,6,1,Early Planet Formation in Embedded Disks. We p...,early planet formation in embedded disks we pr...,early planet formation embedded disks propose ...,early planet formation in embe

In [ ]:
measurements = pd.read_csv('nrao_measurements_use.csv')


In [ ]:
measurements.head(10)

In [ ]:
measurements.info()

## TARGET FROM NOAH

In [ ]:
#TARGET FROM NOAH

# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import plotly.express as px

measurements = pd.read_csv('/content/nrao_measurements_use.csv')

measurements = measurements[measurements.diff_freq < 5]
lines = measurements.query('fs_type == "line"')
lines = lines[['project_code', 'diff_freq', 'med_freq']]
len(lines)

In [ ]:
len(sorted(lines['diff_freq'].round(2).unique()))

In [ ]:
diffs = sorted(lines['diff_freq'].round(2).unique())
meds = sorted(lines['med_freq'].round(2).unique())

In [ ]:
def group_values_by_range(lst, group_size):
    lst.sort()  # Sort the list first
    ranges = []
    start = lst[0]  # Start with the minimum value

    while start < lst[-1]:
        end = start + group_size
        values_in_range = [val for val in lst if start <= val < end]
        ranges.append([start, end])

        # Move to the next value that's greater than the current end
        start = next((val for val in lst if val >= end), lst[-1] + 1)

    return ranges

def build_range(med_vals, diff_vals=0.2, remove_imp= True):
    '''
    Returns ranges with the range set by med_vals and diff_vals
    '''
    diff_range = group_values_by_range(diffs, diff_vals)
    med_range = group_values_by_range(meds, med_vals)

    ranges = []
    for item1 in diff_range:
        for item2 in med_range :
            ranges.append([item1, item2])

    return ranges

In [ ]:
def target_discretization(df, ranges):
    '''
    This function creates a new dataframe with project code and a new target variable vector,
    which will represent the target probabilities of a given project code
    to use for a machine learning model. It will discretize a large range of values (in this case, frequencies)
    into categories set in the ranges parameter.

    param df, pandas.dataframe: dataframe of measurements

    param ranges, 2- D array: Nested list of ranges to sort values into for each project. The length
    of the first dimension of this list will determine the length of the truth vector returned.
    [[[0.05, 0.25], [36.08, 41.08]],]
    '''
    df_new = pd.DataFrame(columns=['project_code', 'target']) # new df to return
    for pc in df.project_code.unique(): # loop through all line projects
        truth_vals = [0 for _ in range(len(ranges))] # create initial truth value list
        df_small = df[df['project_code'] == pc] # subset to correct observations
        for i in range(len(df_small)): # loop through all oobservations
            diff_f = df_small.iloc[i]['diff_freq']
            med_f = df_small.iloc[i]['med_freq']
            for a in range(len(ranges)): # Loop through ranges and match observation to range
                if diff_f >= ranges[a][0][0] and diff_f < ranges[a][0][1] and \
                med_f >= ranges[a][1][0] and med_f < ranges[a][1][1]:
                    truth_vals[a] = truth_vals[a] + 1 # add 1 for each observation in given range
        pos = sum(truth_vals) # now we change to probabilities
        for a in range(len(truth_vals)):
            if truth_vals[a] != 0:
                truth_vals[a] = truth_vals[a]/pos
        df_new.loc[len(df_new)] = [pc, truth_vals] # append to return dataframe

    return df_new

In [ ]:
target = target_discretization(lines, build_range(5))

In [ ]:
target.iloc[0]['target']

In [ ]:
target['target']

In [ ]:
lines[lines['project_code'] == '2016.1.00854.S']

In [ ]:
target.iloc[123]['target'].index(1)

### TARGET: a vector representing the discretized frequency setup

In [ ]:
import pandas as pd

measurements = pd.read_csv('/content/nrao_measurements_use.csv')

# Preprocessing steps to filter the measurements and prepare the target variable
measurements = measurements[measurements.diff_freq < 5]
lines = measurements.query('fs_type == "line"')

# Function to build range and target discretization is already defined

# Assuming build_range and target_discretization functions are defined as provided
ranges = build_range(5)  # Example range building with med_val = 5
target_df = target_discretization(lines, ranges)

# Combine project title and abstract into a single text column for the input
# Assuming the project title and abstract are in 'measurements' DataFrame
measurements['text'] = measurements['project_title'] + " " + measurements['project_abstract']

# Merging the target vector into the measurements DataFrame
# This involves ensuring each project code in 'measurements' matches its corresponding target vector in 'target_df'
final_df = measurements.merge(target_df, on='project_code', how='inner')

# The final_df DataFrame now contains the text data and the corresponding target variable vector for each project code
final_df.head()


In [ ]:
print(final_df.head())

This code block performs tokenization of the dataset text data, which will be the input for the BERT.

Adjust number of tokens accordingly

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Replace NaN values in the 'text' column with an empty string
final_df['text'] = final_df['text'].fillna('')

# Tokenize all of the sentences and map the tokens to thier word IDs
input_ids = []
attention_masks = []
# ma
# For every sentence...
for sent in final_df.text:
    # `encode_plus` will:
    #   (1) Tokenize the sentence
    #   (2) Prepend the `[CLS]` token to the start
    #   (3) Append the `[SEP]` token to the end
    #   (4) Map tokens to their IDs
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 250,           # Pad & truncate all sentences
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks
                        return_tensors = 'pt',     # Return pytorch tensors
                   )

    # Add the encoded sentence to the list
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding)
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(final_df['target_y'].tolist())

print('Original: ', final_df.text[0])
print('Token IDs:', input_ids[0])


CLS and SEP are used in BERT models as markers to mark the beginning and the end of a sentence

In [ ]:
# Splitting Data by 25%
# Masks in BERT models indicate which tokens to pay attention to

# train_inputs, validation_inputs: Tokenized and encoded text data split into training and validation sets.
# train_labels, validation_labels: Corresponding labels for training and validation sets, representing the target frequency setups.
# train_masks, validation_masks: Attention masks for training and validation sets to indicate which tokens the model should focus on.


from sklearn.model_selection import train_test_split

# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2018, test_size=0.25)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=2018, test_size=0.25)

print("Training Set:", len(train_inputs))
print("Validation Set:", len(validation_inputs))


In [ ]:
input_ids

Import DataLoaders to manage data during the training and evaluation phases, handling tasks like shuffling the data for training and creating iterable mini-batches

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch

batch_size = 32

# Create the DataLoader for the training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for the validation set
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

print("Dataloaders created for training and validation sets.")


### NOW WE LOAD THE PRE-TRAINED BERT MODEL

In [ ]:
num_classes = train_labels.shape[1]
num_classes

In [ ]:
adjusted_learning_rate = 5e-5
num_training_steps = len(train_dataloader) * epochs
num_warmup_steps = num_training_steps * 0.1

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
# model = BertForSequenceClassification.from_pretrained(
#     "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = 854, # The number of output labels--2 for binary classification.
#                                         # You can increase this for multi-class tasks.
#     output_attentions = False, # Whether the model returns attentions weights.
#     output_hidden_states = False, # Whether the model returns all hidden-states.
# )

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_classes,
    output_attentions=False,
    output_hidden_states=False,
)

# Tell pytorch to run this model on the GPU.
model.cuda()

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 4

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

### RUN MODEL

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1)
    labels_flat = np.argmax(labels, axis=1)
    return np.mean(pred_flat == labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Now, before you start your training loop, define a starting time:
t0 = time.time()


In [ ]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


def train_model(model, train_dataloader, epochs=4):
    model.train()  # Put the model in training mode
    for epoch_i in range(epochs):
        print(f'\n======== Epoch {epoch_i + 1} / {epochs} ========')
        print('Training...')
        total_loss = 0
        total_accuracy = 0  # Initialize total accuracy for the epoch

        for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
            b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
            model.zero_grad()

            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            # Calculate accuracy
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            total_accuracy += flat_accuracy(logits, label_ids)

        avg_train_loss = total_loss / len(train_dataloader)
        avg_train_accuracy = total_accuracy / len(train_dataloader)  # Calculate the average accuracy over all batches

        print(f"  Average training loss: {avg_train_loss:.2f}")
        print(f"  Average training accuracy: {avg_train_accuracy:.2f}")
        # Optionally, you can add code here to track and print other metrics

In [ ]:
def validate_model(model, validation_dataloader):
    print("\nRunning Validation...")
    model.eval()  # Put the model in evaluation mode
    eval_accuracy = 0
    nb_eval_steps = 0
    for batch in validation_dataloader:
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    print(f"  Accuracy: {eval_accuracy/nb_eval_steps:.2f}")
    # Optionally, add code here to calculate and print other metrics such as F1 score, if desired


In [ ]:
# Train the model
train_model(model, train_dataloader, epochs=4)

In [ ]:
# Train the model
validate_model(model, validation_dataloader, epochs=4)